<a href="https://colab.research.google.com/github/GuadalupeVargas07/EDA/blob/main/repaso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import graphviz
import heapq

class Graph:
    def __init__(self):
        self.graph = {}

    def add_vertex(self, vertex):
        if vertex not in self.graph:
            self.graph[vertex] = {}

    def add_edge(self, vertex1, vertex2, weight):
        if vertex1 in self.graph and vertex2 in self.graph:
            self.graph[vertex1][vertex2] = weight
            self.graph[vertex2][vertex1] = weight

    def remove_vertex(self, vertex):
        if vertex in self.graph:
            del self.graph[vertex]
            for v in self.graph:
                if vertex in self.graph[v]:
                    del self.graph[v][vertex]

    def generate_graphviz_image(self, filename="graph.png", highlight_nodes=None, highlight_edges=None):
        dot = graphviz.Graph(comment='Weighted Undirected Graph')
        highlight_nodes = highlight_nodes or []
        highlight_edges = highlight_edges or []

        # Añadir nodos al grafo con color destacado si están en highlight_nodes
        for vertex in self.graph:
            if vertex in highlight_nodes:
                dot.node(str(vertex), color="red", style="filled", fillcolor="yellow")
            else:
                dot.node(str(vertex))

        # Añadir aristas al grafo, destacando si están en highlight_edges
        for vertex1 in self.graph:
            for vertex2, weight in self.graph[vertex1].items():
                if vertex1 < vertex2:
                    if (vertex1, vertex2) in highlight_edges or (vertex2, vertex1) in highlight_edges:
                        dot.edge(str(vertex1), str(vertex2), label=str(weight), color="blue", penwidth="2")
                    else:
                        dot.edge(str(vertex1), str(vertex2), label=str(weight))

        dot.render(filename, format='png', view=True)

    def sequential_search(self, target):
        found = False
        for vertex in self.graph:
            if vertex == target:
                found = True
                break
        # Generar imagen con el nodo encontrado resaltado
        self.generate_graphviz_image(highlight_nodes=[target] if found else [])
        return f"{target} {'found' if found else 'not found'}."

    def binary_search(self, target):
        vertices = sorted(self.graph.keys())
        left, right = 0, len(vertices) - 1
        found = False
        while left <= right:
            mid = (left + right) // 2
            if vertices[mid] == target:
                found = True
                break
            elif vertices[mid] < target:
                left = mid + 1
            else:
                right = mid - 1
        # Generar imagen con el nodo encontrado resaltado
        self.generate_graphviz_image(highlight_nodes=[target] if found else [])
        return f"{target} {'found' if found else 'not found'}."

    def dfs(self, start):
        visited = set()
        result = []
        total_weight = 0

        def _dfs(vertex):
            nonlocal total_weight
            if vertex not in visited:
                visited.add(vertex)
                result.append(vertex)
                for neighbor, weight in self.graph[vertex].items():
                    if neighbor not in visited:
                        total_weight += weight
                        _dfs(neighbor)

        _dfs(start)
        # Generar imagen con el recorrido DFS resaltado
        self.generate_graphviz_image(highlight_nodes=result)
        return result, total_weight

    def bfs(self, start):
        visited = set([start])
        queue = [start]
        result = []
        total_weight = 0

        while queue:
            vertex = queue.pop(0)
            result.append(vertex)
            for neighbor, weight in self.graph[vertex].items():
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append(neighbor)
                    total_weight += weight

        # Generar imagen con el recorrido BFS resaltado
        self.generate_graphviz_image(highlight_nodes=result)
        return result, total_weight

    def dijkstra(self, start, end=None):
        distances = {vertex: float('infinity') for vertex in self.graph}
        previous = {vertex: None for vertex in self.graph}
        distances[start] = 0
        priority_queue = [(0, start)]

        while priority_queue:
            current_distance, current_vertex = heapq.heappop(priority_queue)
            if current_distance > distances[current_vertex]:
                continue
            for neighbor, weight in self.graph[current_vertex].items():
                distance = current_distance + weight
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    previous[neighbor] = current_vertex
                    heapq.heappush(priority_queue, (distance, neighbor))

        # Si se especifica un nodo de fin, construye el camino y calcula el peso total
        if end:
            path = []
            total_weight = 0
            current = end
            while current is not None:
                path.insert(0, current)
                if previous[current] is not None:
                    total_weight += self.graph[current][previous[current]]
                current = previous[current]
            edges = [(path[i], path[i+1]) for i in range(len(path)-1)]
            self.generate_graphviz_image(highlight_nodes=path, highlight_edges=edges)
            return distances, path, total_weight
        else:
            return distances


def main_menu(graph):
    while True:
        print("\n--- Menu ---")
        print("1. Búsqueda secuencial de un nodo")
        print("2. Búsqueda binaria de un nodo")
        print("3. Búsqueda en profundidad (DFS)")
        print("4. Búsqueda en anchura (BFS)")
        print("5. Encontrar rutas eficientes usando Dijkstra")
        print("6. Visualizar grafo")
        print("7. Salir")
        choice = input("Elige una opción: ")

        if choice == "1":
            target = input("Ingresa el nodo a buscar (secuencial): ")
            print(graph.sequential_search(target))

        elif choice == "2":
            target = input("Ingresa el nodo a buscar (binaria): ")
            print(graph.binary_search(target))

        elif choice == "3":
            start = input("Ingresa el nodo de inicio para DFS: ")
            recorrido, total = graph.dfs(start)
            print("Recorrido DFS:", recorrido)
            print("Peso total del recorrido DFS:", total)

        elif choice == "4":
            start = input("Ingresa el nodo de inicio para BFS: ")
            recorrido, total = graph.bfs(start)
            print("Recorrido BFS:", recorrido)
            print("Peso total del recorrido BFS:", total)

        elif choice == "5":
          start = input("Ingresa el nodo de inicio para Dijkstra: ")
          end = input("Ingresa el nodo de fin (o presiona Enter para omitir): ")
          if end:
            distances, path, total = graph.dijkstra(start, end)
            print(f"Ruta más corta de {start} a {end}: {path} con distancia total {distances[end]} y peso total {total}")
          else:
              distances = graph.dijkstra(start)
              print("Distancias desde", start, ":", distances)


        elif choice == "6":
            graph.generate_graphviz_image()

        elif choice == "7":
            print("Saliendo...")
            break

        else:
            print("Opción no válida. Intenta de nuevo.")


# Crear una instancia del grafo
graph = Graph()

# Agregar vértices
vertices = ["PT", "A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "N", "O", "P", "Q", "R", "S"]
for vertex in vertices:
    graph.add_vertex(vertex)

# Agregar aristas con pesos
edges = [
    ('PT', 'A', 3), ('PT', 'O', 6), ('PT', 'E', 5),
    ('A', 'B', 3), ('A', 'E', 4), ('A', 'I', 2),
    ('B', 'C', 3), ('C', 'K', 3), ('C', 'D', 2),
    ('C', 'J', 6), ('D', 'K', 4), ('D', 'R', 4),
    ('D', 'P', 3), ('D', 'N', 4), ('E', 'H', 3),
    ('E', 'F', 3), ('F', 'G', 4), ('F', 'S', 2),
    ('G', 'J', 3), ('G', 'N', 5), ('H', 'F', 4),
    ('I', 'J', 6), ('J', 'N', 5), ('J', 'D', 5),
    ('K', 'L', 4), ('O', 'T', 2), ('R', 'Q', 3)
]
for vertex1, vertex2, weight in edges:
    graph.add_edge(vertex1, vertex2, weight)

# Ejecutar el menú
main_menu(graph)



--- Menu ---
1. Búsqueda secuencial de un nodo
2. Búsqueda binaria de un nodo
3. Búsqueda en profundidad (DFS)
4. Búsqueda en anchura (BFS)
5. Encontrar rutas eficientes usando Dijkstra
6. Visualizar grafo
7. Salir
C found.

--- Menu ---
1. Búsqueda secuencial de un nodo
2. Búsqueda binaria de un nodo
3. Búsqueda en profundidad (DFS)
4. Búsqueda en anchura (BFS)
5. Encontrar rutas eficientes usando Dijkstra
6. Visualizar grafo
7. Salir
